In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
data = engine.execute("SELECT * FROM measurement")
data

In [7]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [8]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [11]:
# Find the most recent date in the data set.
import pandas as pd
pd.read_sql_query("SELECT date FROM measurement ORDER BY date DESC LIMIT 1;", engine)

,date
0,2017-08-23


In [12]:
data = engine.execute("SELECT date FROM measurement ORDER BY date DESC LIMIT 1")
datez = [list(x) for x in data]
most_recent_date = datez[0][0]
most_recent_date

'2017-08-23'

In [13]:
type(most_recent_date)

str

In [15]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
last_twelve_months_date = dt.datetime.strptime(most_recent_date, '%Y-%m-%d') - dt.timedelta(days=365)
# last_twelve_months_date

# Perform a query to retrieve the data and precipitation scores
prec_data = session.query(Measurement.date,func.avg(Measurement.prcp)).\
                    filter(Measurement.date >= last_twelve_months_date).\
                    group_by(Measurement.date).all()
# prec_data

# Save the query results as a Pandas DataFrame and set the index to the date column
prec_df = pd.DataFrame(prec_data, columns=['Date', 'Precipitation'])
prec_df.set_index('Date', inplace=True)
# prec_df.head()

# Sort the dataframe by date
prec_df = prec_df.sort_index(ascending=True)

In [19]:
prec_df = prec_df.set_index(pd.to_datetime(prec_df.index))
prec_df

,Precipitation
Date,
2016-08-24,1.555000
2016-08-25,0.077143
2016-08-26,0.016667
2016-08-27,0.064000
2016-08-28,0.516667
...,...
2017-08-19,0.030000
2017-08-20,0.005000
2017-08-21,0.193333


In [20]:
prec_df.index[0]

Timestamp('2016-08-24 00:00:00')

In [17]:
type(prec_df.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [28]:
# Use Pandas Plotting with Matplotlib to plot the data
import hvplot.pandas

prec_plot = prec_df.hvplot.bar(title='Hawaii Precipitation 8-23-2017 to 8-24-2018',
                               ylabel='Precipitation (in)',
                                   grid=True)
plt.savefig("Output/Precipitation.png")
prec_plot

:Bars   [Date]   (Precipitation)

<Figure size 432x288 with 0 Axes>

In [30]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prec_df.describe()

,Precipitation
count,365.000000
mean,0.169987
std,0.295722
min,0.000000
25%,0.008571
50%,0.070000
75%,0.191667
max,2.380000


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()